In [75]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [76]:
data = pd.read_csv(r"C:\Users\furka\Desktop\ain4311\project\real-time-motion-detector\data\geophone-sensor-data.csv")

In [77]:
# Özellikler ve etiketleri ayırma
features = data[['mean', 'top_3_mean', 'min', 'max', 'std_dev', 'median', 'q1', 'q3', 'skewness', 'dominant_freq', 'energy']]
labels = data['activity']

# Etiketleri encode etme
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Özellikleri normalize etme
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Veriyi zaman adımlarına bölme
def create_sequences(data, labels, time_steps=10):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(labels[i+time_steps])
    return np.array(X), np.array(y)

time_steps = 6
X, y = create_sequences(features_scaled, labels_encoded, time_steps)

# Veriyi eğitim ve test setlerine bölme
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [70]:
# Özellikler ve etiketleri ayırma
features = data[['mean', 'top_3_mean', 'min', 'max', 'std_dev', 'median', 'q1', 'q3', 'skewness', 'dominant_freq', 'energy']]
labels = data['name']  # "name" sütununu hedef değişken olarak seçiyoruz

# Etiketleri encode etme
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)  # İsimleri sayısal değerlere dönüştür

# Özellikleri normalize etme
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Veriyi zaman adımlarına bölme
def create_sequences(data, labels, time_steps=10):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(labels[i+time_steps])
    return np.array(X), np.array(y)

time_steps = 6
X, y = create_sequences(features_scaled, labels_encoded, time_steps)

# Veriyi eğitim ve test setlerine bölme
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [78]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# LSTM modeli oluşturma
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(np.unique(y)), activation='softmax'))  # Sınıf sayısı kadar nöron ve softmax aktivasyonu

# Modeli derleme
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50


C:\Users\furka\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.4263 - loss: 1.0706 - val_accuracy: 0.5331 - val_loss: 1.0189
Epoch 2/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5371 - loss: 0.9861 - val_accuracy: 0.6237 - val_loss: 0.9159
Epoch 3/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6373 - loss: 0.8970 - val_accuracy: 0.6202 - val_loss: 0.8281
Epoch 4/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6629 - loss: 0.7861 - val_accuracy: 0.7143 - val_loss: 0.6873
Epoch 5/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7049 - loss: 0.6717 - val_accuracy: 0.7422 - val_loss: 0.5773
Epoch 6/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7362 - loss: 0.5794 - val_accuracy: 0.7840 - val_loss: 0.5079
Epoch 7/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7671 - loss: 0.5141 - val_accuracy: 0.7666 - val_loss: 0.4964
Epoch 8/50
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7933 - loss: 0.4582 - val_accuracy: 0.8153 - val_loss: 0.4931
Ep

In [79]:
from sklearn.metrics import classification_report, confusion_matrix

# Modeli değerlendirme
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Doğruluk Skoru: {test_accuracy}")

# Tahminler ve sınıflandırma raporu
y_pred = np.argmax(model.predict(X_test), axis=1)
print("Sınıflandırma Raporu:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))
print("Karışıklık Matrisi:\n", confusion_matrix(y_test, y_pred))


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9337 - loss: 0.1953
Test Doğruluk Skoru: 0.9442508816719055
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Sınıflandırma Raporu:
               precision    recall  f1-score   support

     running       0.97      0.91      0.94        96
     waiting       0.91      0.97      0.94        99
     walking       0.97      0.96      0.96        92

    accuracy                           0.94       287
   macro avg       0.95      0.94      0.94       287
weighted avg       0.95      0.94      0.94       287

Karışıklık Matrisi:
 [[87  7  2]
 [ 2 96  1]
 [ 1  3 88]]


In [80]:
# Yeni veri için tahmin yapma
new_data = features_scaled[:time_steps]  # Örnek olarak ilk 10 adımı aldık
new_data = np.expand_dims(new_data, axis=0)  # Model girişine uygun hale getirme
prediction = model.predict(new_data)
predicted_name = label_encoder.inverse_transform([np.argmax(prediction)])
print(f"Tahmin edilen isim: {predicted_name[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Tahmin edilen isim: walking


In [81]:
train_loss, train_accuracy = model.evaluate(X_train, y_train)
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Eğitim doğruluğu: {train_accuracy:.4f}")
print(f"Test doğruluğu: {test_accuracy:.4f}")


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9495 - loss: 0.1214
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9337 - loss: 0.1953
Eğitim doğruluğu: 0.9346
Test doğruluğu: 0.9443
